# Fraud Detection Project

## Collaboration Plan
**Participants:** Ryan Tang and Santiago von Straussburg

**Github Page**: https://ryantangmj.github.io

### Overview
We are collaboratively analyzing two fraud datasets to explore fraud patterns, feature importance, and machine learning model evaluation.

- **First dataset:** `Cargo_fraud_only.csv`, obtained from [UCR Database](https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/downloads)
- **Second dataset:** Twelve-month and one-month arrest data for fraud, obtained from [UCR Database](https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/downloads)
- **Third dataset:** Yearly unemployment index by State, obtained from [BLS Database](https://data.bls.gov/lausmap/showMap.jsp)


### Technologies Used
1. **GitHub Repository**: For version control, code collaboration, and final project hosting.
2. **Google Colab/Jupyter Notebooks**: For ETL, EDA, and model development.
3. **Discord**: Primary communication platform for real-time discussions.
   - Weekly meetings at 08:00 PM CST on Thursdays for progress reviews and planning.

### Current Focus
Both team members are currently working together on data exploration, including Extraction, Transformation, and Load (ETL) processes, as well as Exploratory Data Analysis (EDA).

### Roadmap & Milestones
#### Milestone 1 – Initial Dataset Selection & ETL
- Identify datasets. - **Completed**
- Perform initial ETL on datasets. - **Completed**
- Establish a GitHub repository and GitHub Pages site. - **Completed**
- Develop basic statistics and initial graph for dataset understanding. - **Completed**

#### Milestone 2 – Additional ETL & Exploratory Data Analysis
- Continue data cleaning and transformation. - **Completed**
- Conduct comprehensive EDA with 3-5 key graphs. - **Completed**
- Present the first project pitch with initial findings. - **Completed**

#### Deliverable 1 – In-Class Presentation
- Finalize and present a 5-7 slide deck covering problem statement, ETL, EDA, and project progress. - **Completed**

#### Milestone 3 – Model Development & Evaluation
- Select machine learning models (Random Forest, XGBoost, Logistic Regression).
- Begin model training and evaluation.
- Analyze model performance and feature importance.

#### Deliverable 2 – Final Website & Presentation
- Finalize project with the deployment of results to the GitHub Pages site.
- Prepare the final presentation summarizing the project lifecycle.

---

## Project Goals

The goal of this collaborative project is to analyze fraud patterns, identify significant features contributing to fraud, and evaluate various machine learning models for fraud detection. By leveraging two distinct datasets, we aim to develop a deep understanding of fraudulent behavior and build predictive models that will aid in identifying and mitigating fraud across different sectors. Specifically, our objectives are as follows:

### 1. Fraud Pattern Analysis
- **Objective:** Investigate patterns and trends in fraud activities across different sectors, particularly cargo-related fraud and arrest data for fraud cases. This will involve examining how fraudulent activities vary over time and geographic locations, and identifying key factors that influence fraud prevalence.

### 2. Feature Importance Assessment
- **Objective:** Analyze and rank the importance of different features contributing to fraud detection. By evaluating features such as transaction details, timestamps, geographic data, and socio-economic indicators, we aim to pinpoint the key variables that can most accurately predict fraud occurrences.

### 3. Machine Learning Model Development & Evaluation
- **Objective:** Develop and compare multiple machine learning models (Random Forest, XGBoost, Logistic Regression) to identify the most effective model for predicting fraud cases. The models will be evaluated on their performance using metrics such as accuracy, precision, recall, and F1-score to ensure robust fraud detection capabilities.

### 4. Comprehensive Data Analysis
- **Objective:** Through thorough data exploration and analysis, we aim to create a holistic view of fraud activities, utilizing exploratory data analysis (EDA) techniques. This will include visualizing fraud trends, uncovering hidden relationships, and establishing baseline statistics.

### 5. Actionable Insights & Final Presentation
- **Objective:** By the end of the project, we aim to deliver a comprehensive set of insights that can inform decision-making regarding fraud prevention and detection strategies. These findings will be shared through a final presentation and a dedicated project website hosted via GitHub Pages.


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from scipy.stats import linregress
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Load the cargo fraud data
cargo_fraud = pd.read_csv('cargo_fraud_only.csv')

# Check if 'data_year' column exists
if 'data_year' in cargo_fraud.columns:
    # Use 'data_year' as 'Year'
    cargo_fraud['Year'] = cargo_fraud['data_year'].astype(int)
else:
    # Extract 'Year' from 'date_recovered' or another date column
    cargo_fraud['date_recovered'] = pd.to_datetime(cargo_fraud['date_recovered'], errors='coerce')
    cargo_fraud['Year'] = cargo_fraud['date_recovered'].dt.year
    cargo_fraud = cargo_fraud.dropna(subset=['Year'])
    cargo_fraud['Year'] = cargo_fraud['Year'].astype(int)

# Ensure the 'State' column exists and matches the 'State' in 'state_unemployment'
if 'state_name' in cargo_fraud.columns:
    cargo_fraud['State'] = cargo_fraud['state_name']
else:
    # Map state abbreviations to full state names
    state_abbrev_to_name = {
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
        'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
        'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
        'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
        'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
        'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
        'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
        'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
        'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
        'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
        'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
        'WI': 'Wisconsin', 'WY': 'Wyoming'
    }
    cargo_fraud['State'] = cargo_fraud['state_abbr'].map(state_abbrev_to_name)

# Exclude rows with missing 'State' or 'Year'
cargo_fraud = cargo_fraud.dropna(subset=['State', 'Year'])

# Exclude Puerto Rico and other territories
cargo_fraud = cargo_fraud[~cargo_fraud['State'].isin(exclude_states)]

# Display the processed cargo fraud data
print(cargo_fraud[['State', 'Year']].head())



In [ ]:
# Load arrest data
arrest_data_url = "https://raw.githubusercontent.com/ryantangmj/ryantangmj.github.io/main/fraud_onemonth_data.csv"
arrest_data = pd.read_csv(arrest_data_url, index_col=0)
arrest_data.reset_index(drop=True, inplace=True)

# Display the first few rows
arrest_data.head()


In [ ]:
# Load HPI data
hpi_data_url = "https://raw.githubusercontent.com/ryantangmj/ryantangmj.github.io/main/hpi_by_state.csv"
hpi_data = pd.read_csv(hpi_data_url)

# Keep relevant columns
hpi_data = hpi_data[["State", "Year", "HPI"]]

# Filter years between 2012 and 2022
hpi_data = hpi_data[(hpi_data["Year"] >= 2012) & (hpi_data["Year"] <= 2022)].reset_index(drop=True)

hpi_data['State'] = hpi_data['State'].str.lower()
hpi_data['Year'] = hpi_data['Year'].astype(int)

# Display the first few rows
hpi_data.head()


In [64]:
# Load poverty data
poverty_data = pd.read_csv('poverty_data.csv')  # Replace with the actual file path

# Reshape poverty data to long format
id_vars = ['State']
value_vars = [col for col in poverty_data.columns if col != 'State']

poverty_long = pd.melt(poverty_data, id_vars=id_vars, value_vars=value_vars,
                       var_name='Variable', value_name='Value')

poverty_long['Year'] = poverty_long['Variable'].str.extract('(\d{4})', expand=False)
poverty_long['Variable_Name'] = poverty_long['Variable'].str.replace(' \d{4}', '', regex=True)

poverty_pivot = poverty_long.pivot_table(index=['State', 'Year'], columns='Variable_Name', values='Value', aggfunc='first').reset_index()
poverty_pivot.columns.name = None

poverty_pivot['Year'] = poverty_pivot['Year'].astype(int)
poverty_pivot['State'] = poverty_pivot['State'].str.lower()

# Convert numeric columns to float
numeric_cols = ['Total population', 'Number in poverty', 'Percentage poverty']
for col in numeric_cols:
    # Convert the column to string first, handling NaN values
    poverty_pivot[col] = poverty_pivot[col].astype(str).replace('nan', '').str.replace(',', '')
    poverty_pivot[col] = poverty_pivot[col].replace('', pd.NA)
    poverty_pivot[col] = poverty_pivot[col].astype(float)

# Rename columns for clarity
poverty_pivot.rename(columns={
    'Total population': 'Total_Population',
    'Number in poverty': 'Number_in_Poverty',
    'Percentage poverty': 'Poverty_Rate'
}, inplace=True)

# Display the first few rows
print(poverty_pivot.head())


     State  Year  Number_in_Poverty  Poverty_Rate  Total_Population
0  alabama  2012              777.0          16.2            4808.0
1  alabama  2013              891.0          18.5            4807.0
2  alabama  2014              848.0          17.8            4765.0
3  alabama  2015              784.0          16.3            4820.0
4  alabama  2016              782.0          16.2            4821.0


In [ ]:
# Load homelessness data
homelessness_data_url = "https://raw.githubusercontent.com/ryantangmj/ryantangmj.github.io/main/homeless_data.csv"
homelessness_data = pd.read_csv(homelessness_data_url)

# Display the first few rows
homelessness_data.head()


### Education

In [67]:
# --- Process Education Data ---

# Load education data
education_data = pd.read_csv('education.csv')  # Replace with the actual file path

# Reshape education data to long format
id_vars = ['State']
value_vars = [col for col in education_data.columns if col != 'State']

education_long = pd.melt(
    education_data,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='Variable',
    value_name='Value'
)

# Extract 'Year' and 'Variable_Name' from the 'Variable' column
education_long['Year'] = education_long['Variable'].str.extract('(\d{4})', expand=False)
education_long['Variable_Name'] = education_long['Variable'].str.replace(' \d{4}', '', regex=True)

# Pivot the data to have one row per 'State' and 'Year'
education_pivot = education_long.pivot_table(
    index=['State', 'Year'],
    columns='Variable_Name',
    values='Value',
    aggfunc='first'
).reset_index()

# Flatten the columns
education_pivot.columns.name = None

# Convert data types
education_pivot['Year'] = education_pivot['Year'].astype(int)
education_pivot['State'] = education_pivot['State'].str.lower()

# Convert numeric columns to float
numeric_cols = [col for col in education_pivot.columns if col not in ['State', 'Year']]
for col in numeric_cols:
    # Convert to string, remove commas
    education_pivot[col] = education_pivot[col].astype(str).str.replace(',', '')
    # Convert to numeric, coercing errors to NaN
    education_pivot[col] = pd.to_numeric(education_pivot[col], errors='coerce')

# Rename columns as needed
# For example, if you have 'Percentage with Bachelor's Degree'
# education_pivot.rename(columns={
#     "Percentage with Bachelor's Degree": 'Bachelor_Degree_Rate'
# }, inplace=True)

# Display the first few rows
print(education_pivot.head())

  State  Year  2003 Urban Influence Code  2013 Rural-urban Continuum Code  \
0    ak  1970                        NaN                              NaN   
1    ak  1980                        NaN                              NaN   
2    ak  1990                        NaN                              NaN   
3    ak  2000                        NaN                              NaN   
4    ak  2003                       12.0                              NaN   

   2013 Urban Influence Code  2023 Rural-urban Continuum Code  \
0                        NaN                              NaN   
1                        NaN                              NaN   
2                        NaN                              NaN   
3                        NaN                              NaN   
4                        NaN                              NaN   

   Bachelor's degree or higher,  Bachelor's degree or higher,-12  \
0                           NaN                              NaN   
1         

### Clean Up

In [ ]:
# Reset index and check for missing values
cargo_fraud.reset_index(drop=True, inplace=True)
print(cargo_fraud.isnull().sum())


In [ ]:
# Convert 'State Code' to string and strip whitespace
arrest_data['State Code'] = arrest_data['State Code'].astype(str).str.strip()

# Map state codes to state names
state_codes = {
    '50': 'Alaska',
    '01': 'Alabama',
    '03': 'Arkansas',
    '54': 'American Samoa',
    '02': 'Arizona',
    '04': 'California',
    '05': 'Colorado',
    '06': 'Connecticut',
    '52': 'Canal Zone',
    '08': 'District of Columbia',
    '07': 'Delaware',
    '09': 'Florida',
    '10': 'Georgia',
    '55': 'Guam',
    '51': 'Hawaii',
    '14': 'Iowa',
    '11': 'Idaho',
    '12': 'Illinois',
    '13': 'Indiana',
    '15': 'Kansas',
    '16': 'Kentucky',
    '17': 'Louisiana',
    '20': 'Massachusetts',
    '19': 'Maryland',
    '18': 'Maine',
    '21': 'Michigan',
    '22': 'Minnesota',
    '24': 'Missouri',
    '23': 'Mississippi',
    '25': 'Montana',
    '26': 'Nebraska',
    '32': 'North Carolina',
    '33': 'North Dakota',
    '28': 'New Hampshire',
    '29': 'New Jersey',
    '30': 'New Mexico',
    '27': 'Nevada',
    '31': 'New York',
    '34': 'Ohio',
    '35': 'Oklahoma',
    '36': 'Oregon',
    '37': 'Pennsylvania',
    '53': 'Puerto Rico',
    '38': 'Rhode Island',
    '39': 'South Carolina',
    '40': 'South Dakota',
    '41': 'Tennessee',
    '42': 'Texas',
    '43': 'Utah',
    '62': 'Virgin Islands',
    '45': 'Virginia',
    '44': 'Vermont',
    '46': 'Washington',
    '48': 'Wisconsin',
    '47': 'West Virginia',
    '49': 'Wyoming'
}

arrest_data['State'] = arrest_data['State Code'].map(state_codes)

# Display the first few rows
arrest_data.head()


In [ ]:
# Load the unemployment data
unemployment_data = pd.read_csv('Unemployment.csv')

# Identify columns that contain 'Unemployment_rate'
unemployment_rate_cols = [col for col in unemployment_data.columns if 'Unemployment_rate_' in col]

# Melt the DataFrame to long format
unemployment_long = pd.melt(
    unemployment_data,
    id_vars=['State', 'Area_Name'],
    value_vars=unemployment_rate_cols,
    var_name='Year',
    value_name='Unemployment_Rate'
)

# Extract the year from the 'Year' column
unemployment_long['Year'] = unemployment_long['Year'].str.extract('Unemployment_rate_(\d+)', expand=False)
unemployment_long['Year'] = unemployment_long['Year'].astype(int)

# Convert 'Unemployment_Rate' to numeric
unemployment_long['Unemployment_Rate'] = pd.to_numeric(unemployment_long['Unemployment_Rate'], errors='coerce')

# Aggregate at the state level by taking the mean unemployment rate for each state and year
state_unemployment = unemployment_long.groupby(['State', 'Year'])['Unemployment_Rate'].mean().reset_index()

# Exclude territories and focus on continental US
exclude_states = ['Puerto Rico', 'Guam', 'Virgin Islands', 'American Samoa', 'Northern Mariana Islands', 'Alaska', 'Hawaii']
state_unemployment = state_unemployment[~state_unemployment['State'].isin(exclude_states)]

# Display the processed unemployment data
print(state_unemployment.head())

In [ ]:
# Map state abbreviations to full state names
state_abbreviations = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'DC': 'District of Columbia',
    'AS': 'American Samoa',
    'GU': 'Guam',
    'MP': 'Northern Mariana Islands',
    'PR': 'Puerto Rico',
    'VI': 'Virgin Islands'
}

homelessness_data['State'] = homelessness_data['State'].map(state_abbreviations)

# Rename columns for years
new_column_names = {
    col: col.split(', ')[-1].split('-')[0] for col in homelessness_data.columns if 'Change in Total Homelessness' in col
}
homelessness_data.rename(columns=new_column_names, inplace=True)

# Melt the DataFrame to long format
df_homelessness = pd.melt(homelessness_data, id_vars=['State'], var_name='Year', value_name='homeless_rate_change')

# Clean 'homeless_rate_change' column
df_homelessness['homeless_rate_change'].replace(' ', np.nan, inplace=True)
df_homelessness.dropna(inplace=True)
df_homelessness['homeless_rate_change'] = df_homelessness['homeless_rate_change'].str.replace('%', '').astype(float)
df_homelessness['Year'] = df_homelessness['Year'].astype(int)

# Display the first few rows
df_homelessness.head()


In [ ]:
# Filter columns that start with 'Percentage poverty' and include 'State'
filtered_poverty_data = poverty_data[[col for col in poverty_data.columns if col.startswith('Percentage poverty') or col == 'State']]

# Rename columns
new_column_names = {
    col: col.split(' ')[-1] for col in filtered_poverty_data.columns
}
filtered_poverty_data.rename(columns=new_column_names, inplace=True)

# Melt the DataFrame to long format
df_poverty = pd.melt(filtered_poverty_data, id_vars=['State'], var_name='Year', value_name='Percentage Poverty')
df_poverty['Year'] = df_poverty['Year'].astype(int)

# Display the first few rows
df_poverty.head()


### EDA

In [ ]:
# Calculate the proportion of fraud by region
fraud_by_region = cargo_fraud['region_name'].value_counts(normalize=True).sort_values(ascending=False)

# Plot the proportion of fraud occurrences by region
plt.figure(figsize=(14, 8))
sns.barplot(x=fraud_by_region.values, y=fraud_by_region.index, palette="coolwarm", edgecolor="black")
plt.title("Proportion of Fraud Occurrences by Region", fontsize=20, weight="bold")
plt.xlabel("Proportion of Fraud Occurrences", fontsize=14, weight="bold")
plt.ylabel("Region", fontsize=14, weight="bold")
plt.grid(axis='x', linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# Calculate the proportion of fraud by state
fraud_by_state = cargo_fraud['state_name'].value_counts(normalize=True).sort_values(ascending=False).reset_index()
fraud_by_state.columns = ['NAME', 'proportion']

# Load USA shapefile
usa = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip')

# Merge the geopandas file and fraud data
merged = usa.merge(fraud_by_state, on='NAME', how='left')
merged['proportion'] = merged['proportion'].fillna(0)

# Exclude Alaska and Hawaii before plotting
merged = merged[~merged['STUSPS'].isin(['AK', 'HI'])]

# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(18, 12))
merged.plot(
    column='proportion', cmap='coolwarm', linewidth=0.6,
    ax=ax, edgecolor='black', legend=True,
    legend_kwds={
        'label': "Proportion of Fraud Occurrences by State",
        'orientation': "horizontal",
        'shrink': 0.8
    }
)
ax.set_axis_off()
ax.set_title('Proportion of Fraud Occurrences by State', fontsize=20, weight="bold", pad=20)
plt.tight_layout()
plt.show()


In [ ]:
# Calculate the proportion of fraud by race
fraud_by_race = cargo_fraud['offender_race'].value_counts(normalize=True).sort_values(ascending=False)

# Plot the proportion of fraud occurrences by race
plt.figure(figsize=(14, 8))
sns.barplot(x=fraud_by_race.values, y=fraud_by_race.index, palette="viridis", edgecolor="black")
plt.title("Proportion of Fraud Occurrences by Race", fontsize=20, weight="bold")
plt.xlabel("Proportion of Fraud Occurrences", fontsize=14, weight="bold")
plt.ylabel("Race", fontsize=14, weight="bold")
plt.grid(axis='x', linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# Counts by 'Year'
fraud_by_year = cargo_fraud['Year'].value_counts().sort_index()

# Plot
plt.figure(figsize=(14, 8))
sns.lineplot(x=fraud_by_year.index, y=fraud_by_year.values, marker="o", color="#5A9BD5", linewidth=2.5)
plt.title("Fraudulent Transactions by Year", fontsize=20, weight="bold")
plt.xlabel("Year", fontsize=16, weight="bold")
plt.ylabel("Number of Fraudulent Transactions", fontsize=16, weight="bold")
plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# Merge the unemployment data with cargo fraud data
merged_df = pd.merge(cargo_fraud, state_unemployment, on=['State', 'Year'], how='left')

# Display the merged DataFrame
print(merged_df.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Load and Process cargo_fraud Data

# Load the cargo fraud data
cargo_fraud = pd.read_csv('cargo_fraud_only.csv')

# Process cargo_fraud data to extract 'Year' and 'State'

# Check if 'data_year' column exists
if 'data_year' in cargo_fraud.columns:
    cargo_fraud['Year'] = cargo_fraud['data_year'].astype(int)
elif 'date_recovered' in cargo_fraud.columns:
    cargo_fraud['date_recovered'] = pd.to_datetime(cargo_fraud['date_recovered'], errors='coerce')
    cargo_fraud['Year'] = cargo_fraud['date_recovered'].dt.year
else:
    print("No date column found in cargo_fraud data.")
    # Handle the error accordingly
    cargo_fraud['Year'] = pd.NA  # Set 'Year' to missing

# Ensure 'State' column exists
if 'state_name' in cargo_fraud.columns:
    cargo_fraud['State'] = cargo_fraud['state_name'].str.lower()
elif 'state_abbr' in cargo_fraud.columns:
    # Map state abbreviations to full state names
    state_abbrev_to_name = {
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
        'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
        'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
        'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
        'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
        'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
        'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
        'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
        'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
        'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
        'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
        'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia'
    }
    cargo_fraud['State'] = cargo_fraud['state_abbr'].map(state_abbrev_to_name).str.lower()
else:
    print("No state column found in cargo_fraud data.")
    # Handle the error accordingly
    cargo_fraud['State'] = pd.NA  # Set 'State' to missing

# Exclude any rows with missing 'State' or 'Year'
cargo_fraud = cargo_fraud.dropna(subset=['State', 'Year'])

# Convert 'Year' to integer
cargo_fraud['Year'] = cargo_fraud['Year'].astype(int)

# Exclude territories and non-continental states
exclude_states = ['puerto rico', 'guam', 'virgin islands', 'american samoa',
                  'northern mariana islands', 'alaska', 'hawaii']
cargo_fraud = cargo_fraud[~cargo_fraud['State'].isin(exclude_states)]

# Step 2: Aggregate the number of fraud cases per state and year
fraud_counts = cargo_fraud.groupby(['State', 'Year']).size().reset_index(name='Fraud_Count')

# Step 3: Load and Process Unemployment Data

# Load the unemployment data
unemployment_data = pd.read_csv('Unemployment.csv')

# Extract state FIPS codes from 'FIPS_Code'
unemployment_data['State_FIPS'] = unemployment_data['FIPS_Code'] // 1000

# Map state FIPS codes to state names
state_fips_to_name = {
    1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas',
    6: 'California', 8: 'Colorado', 9: 'Connecticut', 10: 'Delaware',
    11: 'District of Columbia', 12: 'Florida', 13: 'Georgia', 15: 'Hawaii',
    16: 'Idaho', 17: 'Illinois', 18: 'Indiana', 19: 'Iowa',
    20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana', 23: 'Maine',
    24: 'Maryland', 25: 'Massachusetts', 26: 'Michigan', 27: 'Minnesota',
    28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska',
    32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico',
    36: 'New York', 37: 'North Carolina', 38: 'North Dakota', 39: 'Ohio',
    40: 'Oklahoma', 41: 'Oregon', 42: 'Pennsylvania', 44: 'Rhode Island',
    45: 'South Carolina', 46: 'South Dakota', 47: 'Tennessee', 48: 'Texas',
    49: 'Utah', 50: 'Vermont', 51: 'Virginia', 53: 'Washington',
    54: 'West Virginia', 55: 'Wisconsin', 56: 'Wyoming'
}
unemployment_data['State_Name'] = unemployment_data['State_FIPS'].map(state_fips_to_name)

# Exclude rows with missing 'State_Name'
missing_states = unemployment_data[unemployment_data['State_Name'].isnull()]
if not missing_states.empty:
    print("Missing state names for the following State_FIPS codes:")
    print(missing_states['State_FIPS'].unique())
    # Exclude rows with missing 'State_Name'
    unemployment_data = unemployment_data[~unemployment_data['State_Name'].isnull()]

# Identify columns that contain 'Unemployment_rate_'
unemployment_rate_cols = [col for col in unemployment_data.columns if 'Unemployment_rate_' in col]

# Melt the DataFrame to long format
unemployment_long = pd.melt(
    unemployment_data,
    id_vars=['State_Name'],
    value_vars=unemployment_rate_cols,
    var_name='Year',
    value_name='Unemployment_Rate'
)

# Extract the year from the 'Year' column
unemployment_long['Year'] = unemployment_long['Year'].str.extract('Unemployment_rate_(\d+)', expand=False)
unemployment_long['Year'] = pd.to_numeric(unemployment_long['Year'], errors='coerce')

# Drop rows with missing 'Year'
unemployment_long = unemployment_long.dropna(subset=['Year'])

# Convert 'Year' to integer
unemployment_long['Year'] = unemployment_long['Year'].astype(int)

# Convert 'Unemployment_Rate' to numeric
unemployment_long['Unemployment_Rate'] = pd.to_numeric(unemployment_long['Unemployment_Rate'], errors='coerce')

# Drop rows with missing 'Unemployment_Rate'
unemployment_long = unemployment_long.dropna(subset=['Unemployment_Rate'])

# Aggregate at the state level by taking the mean unemployment rate for each state and year
state_unemployment = unemployment_long.groupby(['State_Name', 'Year'])['Unemployment_Rate'].mean().reset_index()

# Rename 'State_Name' to 'State' and convert to lowercase
state_unemployment.rename(columns={'State_Name': 'State'}, inplace=True)
state_unemployment['State'] = state_unemployment['State'].str.lower()

# Exclude territories and non-continental states
exclude_states = ['puerto rico', 'guam', 'virgin islands', 'american samoa',
                  'northern mariana islands', 'alaska', 'hawaii']
state_unemployment = state_unemployment[~state_unemployment['State'].isin(exclude_states)]

# Verify exclusion
print("\nUnique States in state_unemployment after exclusion:")
print(sorted(state_unemployment['State'].unique()))

# Step 4: Prepare 'fraud_counts' DataFrame

# Ensure 'State' in fraud_counts is lowercase
fraud_counts['State'] = fraud_counts['State'].str.lower()

# Exclude 'Federal' and any territories if present
fraud_counts = fraud_counts[~fraud_counts['State'].isin(['federal'] + [state.lower() for state in exclude_states])]

# Step 5: Merge DataFrames
analysis_df = pd.merge(fraud_counts, state_unemployment, on=['State', 'Year'], how='left')

# Check for missing values after merging
print("\nMissing values in analysis_df after merging:")
print(analysis_df.isnull().sum())

# Step 6: Proceed to Plot
analysis_df_clean = analysis_df.dropna(subset=['Unemployment_Rate', 'Fraud_Count'])

if analysis_df_clean.empty:
    print("\nNo data available for plotting after cleaning.")
else:
    print("\nData available for plotting.")
    # Plot the relationship
    plt.figure(figsize=(12, 8))
    sns.scatterplot(data=analysis_df_clean, x='Unemployment_Rate', y='Fraud_Count')
    plt.title('Fraud Count vs Unemployment Rate', fontsize=16)
    plt.xlabel('Unemployment Rate (%)', fontsize=14)
    plt.ylabel('Fraud Count', fontsize=14)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Optionally, add a regression line
    plt.figure(figsize=(12, 8))
    sns.regplot(data=analysis_df_clean, x='Unemployment_Rate', y='Fraud_Count', scatter_kws={'s':50}, line_kws={'color':'red'})
    plt.title('Fraud Count vs Unemployment Rate with Regression Line', fontsize=16)
    plt.xlabel('Unemployment Rate (%)', fontsize=14)
    plt.ylabel('Fraud Count', fontsize=14)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Calculate and print the correlation coefficient
    correlation = analysis_df_clean['Unemployment_Rate'].corr(analysis_df_clean['Fraud_Count'])
    print(f"\nCorrelation between Unemployment Rate and Fraud Count: {correlation:.4f}")


In [ ]:
# correlation and P values
from scipy.stats import pearsonr

# Calculate correlation coefficient and p-value
corr_coef, p_value = pearsonr(analysis_df_clean['Unemployment_Rate'], analysis_df_clean['Fraud_Count'])
print(f"Correlation Coefficient: {corr_coef:.4f}")
print(f"P-Value: {p_value:.4f}")


In [ ]:
# Assuming 'cargo_fraud' DataFrame is already loaded and processed as before

# Aggregate the number of fraud cases per state and year
fraud_counts = cargo_fraud.groupby(['State', 'Year']).size().reset_index(name='Fraud_Count')

# Ensure 'State' and 'Year' are of appropriate types
fraud_counts['State'] = fraud_counts['State'].str.lower()
fraud_counts['Year'] = fraud_counts['Year'].astype(int)


### Merging

In [68]:
# Merge Check

In [69]:
# Ensure 'State' and 'Year' columns are in the correct format
def prepare_dataframe(df, state_col='State', year_col='Year'):
    df[state_col] = df[state_col].str.lower()
    df[year_col] = df[year_col].astype(int)
    return df

# Prepare fraud_counts
fraud_counts = prepare_dataframe(fraud_counts)

# Prepare poverty_pivot
poverty_pivot = prepare_dataframe(poverty_pivot)

# Prepare hpi_data
hpi_data = prepare_dataframe(hpi_data)

# Prepare state_unemployment
state_unemployment = prepare_dataframe(state_unemployment)

# Prepare education_pivot (already prepared in previous steps)


In [ ]:
# Merge fraud_counts with poverty_pivot
merged_data = pd.merge(fraud_counts, poverty_pivot, on=['State', 'Year'], how='left')

# Merge with education_pivot
merged_data = pd.merge(merged_data, education_pivot, on=['State', 'Year'], how='left')

# Merge with hpi_data
merged_data = pd.merge(merged_data, hpi_data, on=['State', 'Year'], how='left')

# Merge with state_unemployment
merged_data = pd.merge(merged_data, state_unemployment, on=['State', 'Year'], how='left')

In [72]:
# Check for missing values
print("\nMissing values in merged_data:")
print(merged_data.isnull().sum())

# Define critical columns for analysis
critical_columns = [
    'Fraud_Count', 'Poverty_Rate', 'HPI', 'Unemployment_Rate',
    # Add your education variables here
    # For example:
    # 'Bachelor_Degree_Rate',
    # 'High_School_Only_Percent',
    # 'Less_High_School_Percent',
    # 'Some_College_Associate_Percent'
]

# Drop rows with missing values in critical columns
merged_data_clean = merged_data.dropna(subset=critical_columns)



Missing values in merged_data:
State                                                                    0
Year                                                                     0
Fraud_Count                                                              0
Number_in_Poverty                                                        5
Poverty_Rate                                                             5
Total_Population                                                         5
2003 Urban Influence Code                                              338
2013 Rural-urban Continuum Code                                        338
2013 Urban Influence Code                                              338
2023 Rural-urban Continuum Code                                        338
Bachelor's degree or higher,                                           338
Bachelor's degree or higher,-12                                        338
Bachelor's degree or higher,-22                                     

In [73]:
# Assuming 'merged_data' is your merged DataFrame from previous steps

# 1. Handle Missing Values
# Drop rows with any missing values to ensure complete data for modeling
merged_data_clean = merged_data.dropna()

# Verify the number of rows after dropping
print(f"Number of rows before dropping missing values: {merged_data.shape[0]}")
print(f"Number of rows after dropping missing values: {merged_data_clean.shape[0]}")

# 2. Select All Features
# Define the target variable
target = 'Fraud_Count'

# Define features: all columns except 'State', 'Year', and 'Fraud_Count'
features = [col for col in merged_data_clean.columns if col not in ['State', 'Year', 'Fraud_Count']]

# Display selected features
print("\nSelected Features:")
print(features)

# 3. Prepare Data for Modeling
# Extract feature matrix (X) and target vector (y)
X = merged_data_clean[features]
y = merged_data_clean[target]

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Train the CART Model
# Initialize the Decision Tree Regressor
dt_model = DecisionTreeRegressor(random_state=42)

# Train the model
dt_model.fit(X_train, y_train)

# 5. Evaluate the Model
# Make predictions on the test set
y_pred = dt_model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print("\nDecision Tree Regressor Performance:")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.4f}")

# 6. Determine Feature Importance
# Get feature importances from the model
importances = dt_model.feature_importances_

# Create a DataFrame for feature importances
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display feature importances
print("\nFeature Importances:")
print(feature_importance_df)

# Plot feature importances
plt.figure(figsize=(12, 8))
sns.barplot(data=feature_importance_df, x='Importance', y='Feature', palette='viridis')
plt.title('Feature Importances from Decision Tree Regressor', fontsize=16)
plt.xlabel('Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.tight_layout()
plt.show()

Number of rows before dropping missing values: 338
Number of rows after dropping missing values: 0

Selected Features:
['Number_in_Poverty', 'Poverty_Rate', 'Total_Population', '2003 Urban Influence Code', '2013 Rural-urban Continuum Code', '2013 Urban Influence Code', '2023 Rural-urban Continuum Code', "Bachelor's degree or higher,", "Bachelor's degree or higher,-12", "Bachelor's degree or higher,-22", 'Four years of college or higher,', 'High school diploma only,', 'High school diploma only,-12', 'High school diploma only,-22', 'Less than a high school diploma,', 'Less than a high school diploma,-12', 'Less than a high school diploma,-22', 'Percent of adults completing four years of college or higher,', 'Percent of adults completing some college (1-3 years),', "Percent of adults completing some college or associate's degree,", "Percent of adults completing some college or associate's degree,-12", "Percent of adults completing some college or associate's degree,-22", "Percent of adult

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.